# Airline On-Time Arrivals Project
Use the US Dept. of Transportation on-time arrival data for non-stop domestic flights by major air carriers to predict arrival delays.
### Objectives:
1) Build a binary classification model for predicting arrival delays
2) Build a regression model that predicts the extent of the delay
<i>NB: Do not use departure delay as an input feature.</i>
## Data
Data download from: http://transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time
### Variables:
#### Time Period [0:5]
'YEAR','QUARTER','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','FL_DATE'
#### Airline [6:10]
'UNIQUE_CARRIER','AIRLINE_ID','CARRIER','TAIL_NUM','FL_NUM'
#### Origin [11:19]
'ORIGIN_AIRPORT_ID','ORIGIN_AIRPORT_SEQ_ID','ORIGIN_CITY_MARKET_ID',
'ORIGIN','ORIGIN_CITY_NAME','ORIGIN_STATE_ABR','ORIGIN_STATE_FIPS',
'ORIGIN_STATE_NM','ORIGIN_WAC'
#### Destination [20:28]
'DEST_AIRPORT_ID','DEST_AIRPORT_SEQ_ID','DEST_CITY_MARKET_ID','DEST',
'DEST_CITY_NAME','DEST_STATE_ABR','DEST_STATE_FIPS','DEST_STATE_NM',
'DEST_WAC'
#### Departure Performance [29:37]
'CRS_DEP_TIME','DEP_TIME','DEP_DELAY','DEP_DELAY_NEW','DEP_DEL15',
'DEP_DELAY_GROUP','DEP_TIME_BLK','TAXI_OUT','WHEELS_OFF'
#### Arrival Performance [38:46]
'WHEELS_ON','TAXI_IN','CRS_ARR_TIME','ARR_TIME','ARR_DELAY',
'ARR_DELAY_NEW','ARR_DEL15','ARR_DELAY_GROUP','ARR_TIME_BLK'
#### Cancellations and Diversions [47:49]
'CANCELLED','CANCELLATION_CODE','DIVERTED'
#### Flight Summaries [50:55]
'CRS_ELAPSED_TIME','ACTUAL_ELAPSED_TIME','AIR_TIME','FLIGHTS',
'DISTANCE','DISTANCE_GROUP'
#### Cause of Delay [56:60]
'CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY',
'LATE_AIRCRAFT_DELAY'
#### Gate Return Information at Origin Airport [61:63]
'FIRST_DEP_TIME','TOTAL_ADD_GTIME','LONGEST_ADD_GTIME'
#### Diverted Airport Information [64:83] <i>mostly NaN</i>
'DIV_AIRPORT_LANDINGS','DIV_REACHED_DEST','DIV_ACTUAL_ELAPSED_TIME',
'DIV_ARR_DELAY','DIV_DISTANCE','DIV1_AIRPORT','DIV1_AIRPORT_ID',
'DIV1_AIRPORT_SEQ_ID','DIV1_WHEELS_ON','DIV1_TOTAL_GTIME',
'DIV1_LONGEST_GTIME','DIV1_WHEELS_OFF','DIV1_TAIL_NUM','DIV2_AIRPORT',
'DIV2_AIRPORT_ID','DIV2_AIRPORT_SEQ_ID','DIV2_WHEELS_ON',
'DIV2_TOTAL_GTIME','DIV2_LONGEST_GTIME','DIV2_WHEELS_OFF','DIV2_TAIL_NUM',

<i>[84:109] All NaN</i>

In [72]:
# Import libruaries and set environment
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [2]:
# import data
data1 = pd.read_csv('/Volumes/Data/Python Projects/Flight_delay/Data/103642275_T_ONTIME-1_14.csv')

/Users/calvincheung/anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
# Make subset and clean data
df = data1.ix[:,:61]
df = df[df.CANCELLED != 1]
df.shape

(441097, 61)

In [53]:
# Create new columns for delay types
# Delay_Carrier
df['Delay_Carrier'] = float('NaN')
df.loc[df['CARRIER_DELAY'] > 0, 'Delay_Carrier'] = 1
df.loc[df['CARRIER_DELAY'] == 0, 'Delay_Carrier'] = 0
df.loc[pd.isnull(df['CARRIER_DELAY']),'Delay_Carrier'] = 0

# Delay_Weather
df['Delay_Weather'] = float('NaN')
df.loc[df['WEATHER_DELAY'] > 0, 'Delay_Weather'] = 1
df.loc[df['WEATHER_DELAY'] == 0, 'Delay_Weather'] = 0
df.loc[pd.isnull(df['WEATHER_DELAY']),'Delay_Weather'] = 0

# Delay_NAS
df['Delay_NAS'] = float('NaN')
df.loc[df['NAS_DELAY'] > 0, 'Delay_NAS'] = 1
df.loc[df['NAS_DELAY'] == 0, 'Delay_NAS'] = 0
df.loc[pd.isnull(df['NAS_DELAY']),'Delay_NAS'] = 0

# Delay_Security
df['Delay_Security'] = float('NaN')
df.loc[df['SECURITY_DELAY'] > 0, 'Delay_Security'] = 1
df.loc[df['SECURITY_DELAY'] == 0, 'Delay_Security'] = 0
df.loc[pd.isnull(df['SECURITY_DELAY']),'Delay_Security'] = 0

# Delay_LateAircraft
df['Delay_LateAircraft'] = float('NaN')
df.loc[df['LATE_AIRCRAFT_DELAY'] > 0, 'Delay_LateAircraft'] = 1
df.loc[df['LATE_AIRCRAFT_DELAY'] == 0, 'Delay_LateAircraft'] = 0
df.loc[pd.isnull(df['LATE_AIRCRAFT_DELAY']),'Delay_LateAircraft'] = 0

# Delay_Any
df['Delay_Any'] = float(0)
df.loc[df['Delay_Carrier'] == 1, 'Delay_Any'] = 1
df.loc[df['Delay_Weather'] == 1, 'Delay_Any'] = 1
df.loc[df['Delay_NAS'] == 1, 'Delay_Any'] = 1
df.loc[df['Delay_Security'] == 1, 'Delay_Any'] = 1
df.loc[df['Delay_LateAircraft'] == 1, 'Delay_Any'] = 1


In [66]:
df.ix[:10,41:]

,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Delay_Carrier,Delay_Weather,Delay_NAS,Delay_Security,Delay_LateAircraft,Delay_Any,UC_number
0,1238,13,13,0,0,1200-1259,0,NaN,0,385,...,NaN,NaN,NaN,0,0,0,0,0,0,7
1,1226,1,1,0,0,1200-1259,0,NaN,0,385,...,NaN,NaN,NaN,0,0,0,0,0,0,7
3,1324,59,59,1,3,1200-1259,0,NaN,0,385,...,0,0,0,0,1,0,0,0,1,7
4,1415,110,110,1,7,1200-1259,0,NaN,0,385,...,0,0,0,0,1,0,0,0,1,7
5,1217,-8,0,0,-1,1200-1259,0,NaN,0,385,...,NaN,NaN,NaN,0,0,0,0,0,0,7
6,1212,-13,0,0,-1,1200-1259,0,NaN,0,385,...,NaN,NaN,NaN,0,0,0,0,0,0,7
7,1215,-10,0,0,-1,1200-1259,0,NaN,0,385,...,NaN,NaN,NaN,0,0,0,0,0,0,7
8,1204,-21,0,0,-2,1200-1259,0,NaN,0,385,...,NaN,NaN,NaN,0,0,0,0,0,0,7
9,1245,20,20,1,1,1200-1259,0,NaN,0,385,...,0,0,0,0,1,0,0,0,1,7
10,1344,79,79,1,5,1200-1259,0,NaN,0,385,...,79,0,0,0,0,1,0,0,1,7


In [55]:
# Check which variables have relationship with delay cases
print(df['DAY_OF_WEEK'][df['Delay_Any'] == 1].value_counts(normalize = True))
print(df['UNIQUE_CARRIER'][df['Delay_Any'] == 1].value_counts(normalize = True))
print(df['DISTANCE_GROUP'][df['Delay_Any'] == 1].value_counts(normalize = True))
print(df['ORIGIN_STATE_FIPS'][df['Delay_Any'] == 1].value_counts(normalize = True))
print(df['DEST_STATE_FIPS'][df['Delay_Any'] == 1].value_counts(normalize = True))


4    0.201702
5    0.186959
3    0.160825
1    0.117014
6    0.116664
7    0.113456
2    0.103380
Name: DAY_OF_WEEK, dtype: float64
WN    0.241820
EV    0.138432
DL    0.120823
OO    0.092455
MQ    0.081529
UA    0.078254
AA    0.078179
US    0.055753
B6    0.053853
FL    0.021509
F9    0.017501
AS    0.010659
VX    0.005717
HA    0.003517
Name: UNIQUE_CARRIER, dtype: float64
2     0.249062
3     0.204952
4     0.162108
1     0.121498
5     0.109597
7     0.045586
6     0.039610
8     0.020284
10    0.019709
11    0.015201
9     0.012392
Name: DISTANCE_GROUP, dtype: float64
48    0.114606
6     0.102880
17    0.088171
12    0.077937
13    0.068212
8     0.058911
36    0.053794
51    0.030460
34    0.026576
4     0.026435
37    0.026418
26    0.026185
32    0.024668
42    0.022301
24    0.021468
29    0.020659
27    0.019034
25    0.017876
39    0.016959
47    0.016276
49    0.013759
53    0.011584
55    0.011084
22    0.010334
18    0.007684
21    0.007300
41    0.007250
40    0.006450

In [64]:
df['UC_number'] = float('NaN')
df.loc[df['UNIQUE_CARRIER'] == 'WN', 'UC_number'] = 1
df.loc[df['UNIQUE_CARRIER'] == 'EV', 'UC_number'] = 2
df.loc[df['UNIQUE_CARRIER'] == 'DL', 'UC_number'] = 3
df.loc[df['UNIQUE_CARRIER'] == 'OO', 'UC_number'] = 4
df.loc[df['UNIQUE_CARRIER'] == 'MQ', 'UC_number'] = 5
df.loc[df['UNIQUE_CARRIER'] == 'UA', 'UC_number'] = 6
df.loc[df['UNIQUE_CARRIER'] == 'AA', 'UC_number'] = 7
df.loc[df['UNIQUE_CARRIER'] == 'US', 'UC_number'] = 8
df.loc[df['UNIQUE_CARRIER'] == 'B6', 'UC_number'] = 9
df.loc[df['UNIQUE_CARRIER'] == 'FL', 'UC_number'] = 10
df.loc[df['UNIQUE_CARRIER'] == 'F9', 'UC_number'] = 11
df.loc[df['UNIQUE_CARRIER'] == 'AS', 'UC_number'] = 12
df.loc[df['UNIQUE_CARRIER'] == 'VX', 'UC_number'] = 13
df.loc[df['UNIQUE_CARRIER'] == 'HA', 'UC_number'] = 14


In [61]:
# Check NaN
# pd.isnull(df['DEST_STATE_FIPS']).any()

False

In [69]:
# Classification 1 - Decision Tree
# Create the target and features numpy arrays:
target = df['Delay_Any'].values
features = df[['DAY_OF_WEEK','UC_number','DISTANCE_GROUP','ORIGIN_STATE_FIPS','DEST_STATE_FIPS']]

tree1 = tree.DecisionTreeClassifier()
tree1 = tree1.fit(features, target)

# features and score
print(tree1.feature_importances_)
print(tree1.score(features, target))

[ 0.17548616  0.22601195  0.13191615  0.24438369  0.22220205]
0.7463188369


In [73]:
# Classification 2 - Random Forest
# Create the target and features numpy arrays:
# target = df['Delay_Any'].values
# features = df[['DAY_OF_WEEK','UC_number','DISTANCE_GROUP','ORIGIN_STATE_FIPS','DEST_STATE_FIPS']]

forest1 = RandomForestClassifier(n_estimators = 100, random_state = 1)
forest1 = forest1.fit(features, target)
# max_depth = 10, min_samples_split=2, 
# features and score
print(forest1.feature_importances_)
print(forest1.score(features, target))

[ 0.19266471  0.18948513  0.12799537  0.24791553  0.24193926]
0.746300700299


In [12]:
print(df['DAY_OF_WEEK'][df['Delay_Carrier'] == 1].value_counts(normalize = True))
print(df['DAY_OF_WEEK'][df['Delay_Weather'] == 1].value_counts(normalize = True))
print(df['DAY_OF_WEEK'][df['Delay_NAS'] == 1].value_counts(normalize = True))
print(df['DAY_OF_WEEK'][df['Delay_Security'] == 1].value_counts(normalize = True))
print(df['DAY_OF_WEEK'][df['Delay_LateAircraft'] == 1].value_counts(normalize = True))
print(df['DAY_OF_WEEK'][df['Delay_Any'] == 1].value_counts(normalize = True))


4    0.196094
5    0.188762
3    0.154613
1    0.122976
7    0.120801
6    0.117466
2    0.099289
Name: DAY_OF_WEEK, dtype: float64
4    0.208301
5    0.163411
3    0.152644
2    0.143180
6    0.121646
7    0.111314
1    0.099505
Name: DAY_OF_WEEK, dtype: float64
4    0.215437
5    0.180562
3    0.163202
6    0.116352
7    0.113800
1    0.112736
2    0.097912
Name: DAY_OF_WEEK, dtype: float64
7    0.186495
5    0.186495
4    0.180064
6    0.141479
3    0.115756
1    0.109325
2    0.080386
Name: DAY_OF_WEEK, dtype: float64
4    0.208108
5    0.194681
3    0.156699
1    0.119250
6    0.113830
7    0.112579
2    0.094853
Name: DAY_OF_WEEK, dtype: float64
4    0.201702
5    0.186959
3    0.160825
1    0.117014
6    0.116664
7    0.113456
2    0.103380
Name: DAY_OF_WEEK, dtype: float64


In [13]:
print(df['DAY_OF_WEEK'][df['Delay_Carrier'] == 1].value_counts())
print(df['DAY_OF_WEEK'][df['Delay_Weather'] == 1].value_counts())
print(df['DAY_OF_WEEK'][df['Delay_NAS'] == 1].value_counts())
print(df['DAY_OF_WEEK'][df['Delay_Security'] == 1].value_counts())
print(df['DAY_OF_WEEK'][df['Delay_LateAircraft'] == 1].value_counts())
print(df['DAY_OF_WEEK'][df['Delay_Any'] == 1].value_counts())


4    13345
5    12846
3    10522
1     8369
7     8221
6     7994
2     6757
Name: DAY_OF_WEEK, dtype: int64
4    2399
5    1882
3    1758
2    1649
6    1401
7    1282
1    1146
Name: DAY_OF_WEEK, dtype: int64
4    13763
5    11535
3    10426
6     7433
7     7270
1     7202
2     6255
Name: DAY_OF_WEEK, dtype: int64
7    58
5    58
4    56
6    44
3    36
1    34
2    25
Name: DAY_OF_WEEK, dtype: int64
4    14476
5    13542
3    10900
1     8295
6     7918
7     7831
2     6598
Name: DAY_OF_WEEK, dtype: int64
4    24203
5    22434
3    19298
1    14041
6    13999
7    13614
2    12405
Name: DAY_OF_WEEK, dtype: int64


In [ ]:
temp1 = data1.groupby('UNIQUE_CARRIER').DELAY.count()